In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy
import string
import pickle
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import BernoulliNB

from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [18]:
train = pd.read_csv('aclImdb/train_collated.csv')
x_train = train['Text']
y_train = train['Score']

In [19]:
def restrict_labels(labels):
    restricted_labels = []
    for score in labels:
        if score > 0:
            restricted_labels.append(1)
        else:
            restricted_labels.append(0)

    return restricted_labels

def define_labels_with_threshold(labels, threshold=6.5):
    defined_labels = []

    for score in labels:
        if score < threshold:
            defined_labels.append(0)
        else:
            defined_labels.append(1)

    return defined_labels

In [20]:
y_train = restrict_labels(y_train)



In [21]:
nlp = English()
stopwords = list(STOP_WORDS)
punctuations = string.punctuation


In [22]:
def tokenizer(sentence):
    mytokens = nlp(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

def clean_text(text):
    return text.strip().lower()

In [23]:
vectorizer = CountVectorizer(tokenizer = tokenizer, ngram_range=(1,1))
tfvectorizer = TfidfVectorizer(tokenizer = tokenizer)

In [27]:
model = Pipeline([("cleaner", predictors()),
                 ('vectorizer', tfvectorizer),
                 ('classifier', BernoulliNB())])

In [28]:
def train_model(model, X_train, y_train):
    ''' TODO: train your model based on the training data '''
    model.fit(X_train, y_train)


def predict(model, X_test):
    ''' TODO: make your prediction here '''
    return model.predict(X_test)

In [29]:
train_model(model, x_train, y_train)
y_pred = predict(model, x_train)


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
score = f1_score(y_train, y_pred)
print('score on validation for training set: recall=', recall, " precision=", precision, " f1-score=", score)


In [ ]:
test = pd.read_csv('aclImdb/test_collated.csv')
x_test = test['Text']
y_test = test['Score']
y_test = define_labels_with_threshold(y_test)


In [ ]:
y_pred = predict(model, x_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
score = f1_score(y_test, y_pred)

print('score on validation for test set: recall =', recall, " precision =", precision, " f1-score =", score)
